<a href="https://colab.research.google.com/github/mhkarimid/Bioox-/blob/main/ANN%20model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, explained_variance_score
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import load_model


In [2]:
df=pd.read_csv('https://raw.githubusercontent.com/mhkarimid/Data/main/newdata.csv', 
               names=['Time (min)', 'Gold grade (g/ton)', 'Pyrite content (wt.%)',
               'Ore content (wt.%)', 'pH', 'ORP (mV)', 'D-Gal (wt.%)',
               'D-Suc (wt.%)', 'D-FrU (wt.%)', 'Ferric (g/lit)', 'Pyrite dissolution (%)'])
df.head()

,Time (min),Gold grade (g/ton),Pyrite content (wt.%),Ore content (wt.%),pH,ORP (mV),D-Gal (wt.%),D-Suc (wt.%),D-FrU (wt.%),Ferric (g/lit),Pyrite dissolution (%)
0,-0.500000,0.5,0.5,-0.444444,-0.191734,-0.460602,-0.155844,-0.155844,-0.155844,-0.177262,-0.339150
1,-0.456522,0.5,0.5,-0.444444,-0.201303,-0.443605,-0.155844,-0.155844,-0.155844,-0.161934,-0.323117
2,-0.413043,0.5,0.5,-0.444444,-0.244366,-0.432274,-0.155844,-0.155844,-0.155844,-0.165186,-0.327591
3,-0.369565,0.5,0.5,-0.444444,-0.273074,-0.304795,-0.155844,-0.155844,-0.155844,-0.131744,-0.278188
4,-0.326087,0.5,0.5,-0.444444,-0.225227,-0.214143,-0.155844,-0.155844,-0.155844,-0.118739,-0.251342


In [70]:
X=df.drop(['Gold grade (g/ton)','Ferric (g/lit)','Pyrite dissolution (%)'], axis=1).values
y=df[['Pyrite dissolution (%)','Ferric (g/lit)']].values

Rte0=[]
Rte1=[]

Rtr0=[]
Rtr1=[]

RMSEte0=[]
RMSEte1=[]

RMSEtr0=[]
RMSEtr1=[]

MODEL=[]
for i in range(1,19):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05*i)
    
    model=Sequential()
    model.add(Dense(7, activation='sigmoid'))
    model.add(Dense(4, activation='sigmoid'))
    model.add(Dense(2, activation='sigmoid'))
    model.add(Dense(2))
    model.compile(optimizer='adam', loss='mse')
    model.fit(x=X_train,y=y_train, 
              validation_data=(X_test,y_test), 
              verbose=0,
              batch_size=128,
              epochs=1000)
    
    MODEL.append(model)
    
    ytest=model.predict(X_test)
    dk0=pd.DataFrame(list(zip(ytest.transpose()[0],y_test.transpose()[0])))
    dk1=pd.DataFrame(list(zip(ytest.transpose()[1],y_test.transpose()[1])))
    d0=dk0.corr()
    d1=dk1.corr()
    Rte0.append(d0.min()[0])
    Rte1.append(d1.min()[0])
    
    ytrain=model.predict(X_train)
    dk0=pd.DataFrame(list(zip(ytest.transpose()[0],y_test.transpose()[0])))
    dk1=pd.DataFrame(list(zip(ytest.transpose()[1],y_test.transpose()[1])))
    d0=dk0.corr()
    d1=dk1.corr()
    Rtr0.append(d0.min()[0])
    Rtr1.append(d1.min()[0])
    
    
    rmse0=(mean_squared_error(ytest.transpose()[0],y_test.transpose()[0]))
    rmse1=(mean_squared_error(ytest.transpose()[1],y_test.transpose()[1]))
    RMSEte0.append(rmse0)
    RMSEte1.append(rmse1)
    
    
    rmse0=(mean_squared_error(ytrain.transpose()[0],y_train.transpose()[0]))
    rmse1=(mean_squared_error(ytrain.transpose()[1],y_train.transpose()[1]))
    RMSEtr0.append(rmse0)
    RMSEtr1.append(rmse1)
    
    
    
    
Results=pd.DataFrame(list(zip(RMSEtr0, Rtr0, RMSEte0, Rte0, RMSEtr1, Rtr1, RMSEte1, Rte1)), 
                     columns=['RMSE0 (training)', 
                              'Correlation factor0 (training)',
                              'RMSE0 (testing)', 
                              'Correlation factor0 (testing)',
                              'RMSE1 (training)', 
                              'Correlation factor1 (training)',
                              'RMSE1 (testing)', 
                              'Correlation factor1 (testing)'])
Results


10/10 [==============================] - 0s 2ms/step


,RMSE0 (training),Correlation factor0 (training),RMSE0 (testing),Correlation factor0 (testing),RMSE1 (training),Correlation factor1 (training),RMSE1 (testing),Correlation factor1 (testing)
0,0.001770,0.985668,0.001807,0.985668,0.000732,0.989952,0.000695,0.989952
1,0.002248,0.978764,0.002440,0.978764,0.001177,0.983715,0.001187,0.983715
2,0.002834,0.971865,0.003099,0.971865,0.000944,0.984824,0.001125,0.984824
3,0.003142,0.974216,0.002932,0.974216,0.001076,0.981962,0.001373,0.981962
4,0.002180,0.977395,0.002541,0.977395,0.001597,0.974249,0.001780,0.974249
5,0.002793,0.975391,0.002897,0.975391,0.000890,0.985121,0.001087,0.985121
6,0.003562,0.969036,0.003595,0.969036,0.001761,0.973078,0.001877,0.973078
7,0.004235,0.966362,0.003806,0.966362,0.001972,0.971153,0.001899,0.971153
8,0.003087,0.972627,0.003064,0.972627,0.001120,0.983628,0.001173,0.983628
9,0.003042,0.971646,0.003289,0.971646,0.001969,0.971367,0.002126,0.971367
